In [1]:
import json
import random
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from user_agent2 import generate_user_agent
from bs4 import BeautifulSoup as BS

# Function to get user agents
def get_user_agents():
    return [generate_user_agent(navigator="chrome") for _ in range(1, 5000)]

# Function to get a random user agent
def get_random_user_agent(user_agents):
    return random.choice(user_agents[1500:2000])

# Function to scrape data and save to JSON
def scrape_and_save_data(run_number, user_agents):
    # Set up Chrome options with a random user agent and enable headless mode
    chrome_options = Options()
    chrome_options.add_argument(f'user-agent={get_random_user_agent(user_agents)}')
    chrome_options.add_experimental_option('excludeSwitches', ['enable-automation'])

    # Start the WebDriver
    driver = webdriver.Chrome(options=chrome_options)
    actions = ActionChains(driver)
    actions.move_by_offset(10, 10).perform()

    driver.get('https://abrahamjuliot.github.io/creepjs/')
    driver.maximize_window()

    time.sleep(5)

    # Extract data
    fp_id_element = driver.find_elements(by="xpath", value='//div[@class="ellipsis-all"]/span')
    fp_id = ''.join([i.text for i in fp_id_element.find_elements(by="xpath", value=".//span")])

    trust_score = driver.find_element(by="xpath", value='//div[@class="visitor-info"]//div[@class="col-six"][1]/div/span[1]').text
    lies = driver.find_element(by="xpath", value='//div[@class=" lies"]/label').text
    bot = driver.find_element(by="xpath", value='//div[@class="unblurred"][1]').text.split()[1]

    # Take a screenshot of the entire webpage
    screenshot_filename = f'page_{run_number}.png'
    driver.save_screenshot(screenshot_filename)

    # Create a dictionary with the extracted data and screenshot filename
    data_dict = {
        "fp_id": fp_id,
        "trust_score": trust_score,
        "lies": lies,
        "bot": bot,
        "screenshot_filename": screenshot_filename
    }

    # Save the data to a JSON file
    json_filename = f'scraped_data_{run_number}.json'
    with open(json_filename, 'w') as json_file:
        json.dump(data_dict, json_file)

    # Close the WebDriver
    driver.quit()

# Run the script three times
for run_number in range(1, 4):
    user_agents = get_user_agents()
    scrape_and_save_data(run_number, user_agents)
    print(f"Run {run_number} completed.")


Run 1 completed.
Run 2 completed.
Run 3 completed.
